<a href="https://colab.research.google.com/github/professorgilzamir/artificialintelligence/blob/master/BackPropagationFashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from torchvision import datasets as dts
from torchvision.transforms import ToTensor
from matplotlib import pyplot as ptl
import numpy as np

In [4]:
traindt = dts.FashionMNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True
)

testdt = dts.FashionMNIST(
    root = 'data',
    train = False,
    transform = ToTensor()
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
print(traindt[0][0].numpy().shape)
print(testdt[0][0].numpy().shape)

(1, 28, 28)
(1, 28, 28)


Implementa uma rede neural do tipo MLP com as etapas forward e backward (backpropagation).

In [17]:
import numpy as np
from copy import deepcopy

class NeuralNetwork:
    def __init__(self, I=2, H=None, O=2):
        if H is None:
            H = [1]
        self.inputs = I #quantidade de neurônios na camada de entrada.
        self.hiddens = H.copy() #quantidade de neurônios em uma ou mais camada oculta.
        self.outputs = O #quantidade de neurônios na camada de saída
        self.layers = [I] + H + [O] #cria uma lista com a quantidade de neurônio em cada camada, inclusive a entrada e a saída
        self.weights = [] #lista de matrizes de peso. A quantidade de matrizes é igual a n-1, onde n é a quantidade de camadas.
        np.random.seed(0) #inicialização do mesmo conjunto de pesos sempre.
        for i in range(len(self.layers)-1): #laço para criar as matrizes iniciais aleatórias
            size = (self.layers[i+1], self.layers[i]+1)
            self.weights.append(np.random.normal(0, 1.0, size=size))
        self.activation = lambda x : 1/(1+np.exp(-x)) #função de ativação
        self.derivative = lambda x : (self.activation(x) * 
                                        (1 - self.activation(x))) #derivada da função de ativação

    def forward(self, x):
        x = x.copy()
        x.append(1.0)
        #print(self.weights[0].shape)
        self.ib = np.array(x, dtype=np.float32)
        self.layer_inputs = [self.ib]
        self.layer_outputs = [self.ib]
        for idx in range(len(self.layers)-1):
            self.layer_inputs.append(self.weights[idx] @ self.layer_outputs[idx])
            aH = self.activation(self.layer_inputs[idx+1])
            if len(self.layer_outputs) < (len(self.layers) - 1):
                aH = np.append(aH, 1.0)
            self.layer_outputs.append(aH)
        return self.layer_outputs[-1]

    def backward(self, x, y, lr = 0.1):
      assert len(self.layers) > 1
      yl = self.forward(x)
      e = (y - yl)
      deltaHO = []
      dINo = self.derivative(self.layer_inputs[-1])
      deltak = np.zeros( (self.layers[-1], ) , dtype=np.float32)
      for k in range(self.layers[-1]):
          deltak[k] = -e[k] * dINo[k]
          for j in range(self.layers[-2]+1):
              deltaHO.append(deltak[k] * self.layer_outputs[-2][j])
      deltaHO = np.reshape(np.array(deltaHO, dtype=np.float32), 
                              ( self.layers[-1], self.layers[-2] + 1 ) )
      self.weights[-1] -= lr * deltaHO

      l = len(self.layers)-2
      while l > 0:
        deltaj = np.zeros( (self.layers[l], ), dtype=np.float32)
        dINh = self.derivative(self.layer_inputs[l])
        deltaIH = []
        for j in range(self.layers[l]):
            dj = 0
            for k in range(len(deltak)):
                dj = dj + self.weights[l][k, j] * deltak[k]
            deltaj[j] = dj * dINh[j]
            for i in range(self.layers[l-1]+1): #range(self.inputs+1):
                deltaIH.append(deltaj[j] * self.layer_outputs[l-1][i])
        deltaIH = np.reshape(np.array(deltaIH, dtype=np.float32), 
                                (self.layers[l], self.layers[l-1] + 1) )
        self.weights[l-1] -= lr * deltaIH
        deltaHO = deepcopy(deltaIH)
        deltak = deepcopy(deltaj)
        l = l - 1

A rede neural tem que ter pelo menos duas camadas (entrada e saída) e uma quantidade variável de camadas escondidas.

In [12]:
net = NeuralNetwork(2, [2], 2)
#print(net.forward([1, 1]))
net.backward([1, 1], [1, 1])

In [5]:
def encoding_output(v):
    a = np.zeros(10)
    a[v] = 1
    return a

x = traindt[0][0].numpy().reshape((28*28, ))
y = traindt[0][1]
print("ANTES = ", y)
y = encoding_output(y)
print("DEPOIS = ", y)

ANTES =  9
DEPOIS =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [20]:
INPUTS = 28*28
HIDDENS = [20, 20]
OUTPUTS = 10
net = NeuralNetwork(INPUTS, HIDDENS, OUTPUTS)

Utilizamos a estratégia de mini-batches em que uma amostra de tamanho 32 é selecionada aleatoriamente em cada época de treinamento. Realizamos um total de 100 épocas.

In [21]:
EPOCHS = 100
for k in range(EPOCHS):
    samples = np.random.choice( len(traindt), size=32 )
    error = 0
    for s in samples:
        x = list(traindt[s][0].numpy().reshape(28*28))
        y = encoding_output([traindt[s][1]])
        yl = net.forward(x)
        error += sum((y - yl)**2)/10.0
        net.backward(x, y)
    print("ERRO DE TREINO: ", error/len(samples))

ERRO DE TREINO:  0.46889100810033285
ERRO DE TREINO:  0.23923660508000694
ERRO DE TREINO:  0.17556582102970492
ERRO DE TREINO:  0.1157774876354745
ERRO DE TREINO:  0.0899378295949811
ERRO DE TREINO:  0.09162857024173289
ERRO DE TREINO:  0.09060712316374993
ERRO DE TREINO:  0.08884666872957306
ERRO DE TREINO:  0.0883734264473049
ERRO DE TREINO:  0.09004072232467336
ERRO DE TREINO:  0.08696046940722088
ERRO DE TREINO:  0.08430820224758008
ERRO DE TREINO:  0.08797858169597803
ERRO DE TREINO:  0.08666147930623852
ERRO DE TREINO:  0.09130233533612125
ERRO DE TREINO:  0.08922926634205966
ERRO DE TREINO:  0.0873401074788828
ERRO DE TREINO:  0.09022568426114604
ERRO DE TREINO:  0.08216363973788733
ERRO DE TREINO:  0.08699144196365784
ERRO DE TREINO:  0.08829788698649478
ERRO DE TREINO:  0.08785760992513193
ERRO DE TREINO:  0.08912117211061453
ERRO DE TREINO:  0.0892955934896587
ERRO DE TREINO:  0.08841290294048243
ERRO DE TREINO:  0.08802330527081649
ERRO DE TREINO:  0.08224026212963122
ERRO D

O teste deve ser feito em todo o conjunto de dados de teste e não apenas em uma amostra.

In [25]:
error = 0
samples = np.random.choice( len(testdt), size=len(testdt), replace=False)
acertos = 0

for s in samples:
    x = list(testdt[s][0].numpy().reshape(28*28))
    y = encoding_output([testdt[s][1]])
    yl = net.forward(x)

    max_Y = np.argmax(y)
    max_YL = np.argmax(yl)

    if (max_Y == max_YL):
      acertos += 1

    error += sum((y - yl)**2)/10.0
    #net.backward(x, y)
print("ERRO DE TESTE: ", error/len(samples))
print("QTD ACERTOS: ", acertos)
print("PERCENTAGEM DE ACERTO: ", (acertos/len(samples) * 100.0))

ERRO DE TESTE:  0.06651538169564923
QTD ACERTOS:  5033
PERCENTAGEM DE ACERTO:  50.33
